### Application of functions looped over a profile
This is needed in order to: 
- [ ] associate measured points with locations within a profile (with the option to look a little bit upstream and downstream)
- [ ] apply the Lacey functions on each profile
- [ ] apply least squares fitting functions on each profile
- [ ] optimize single parameters over all profiles eventually

Here we focus on one single profile application first.

In [ ]:
import orprofile
import geopandas as gpd
import os
import xugrid as xu
import xarray as xr
import numpy as np

In [ ]:
fn_splines = "../examples/data/splines.geojson"
fn_points = "../examples/data/bamboi_survey.geojson"
splines = gpd.read_file(fn_splines)

mesh = orprofile.api.mesh.Mesh(splines, n=5, m=10, points=fn_points)
mesh


In [ ]:
mesh.plot()

### organizing rows and columns into a XUGridDataSet

In [ ]:
# make a columns and row coordinate for each node
columns = np.repeat([np.arange(mesh.n)], mesh.m, axis=0).flatten()
rows = np.repeat(np.arange(mesh.m), mesh.n)

# prepare UgridDataArrays, using the grid
da_rows = xu.UgridDataArray(
    xr.DataArray(
        data=rows,
        dims=[mesh.mesh2d.face_dimension]
    ),
    grid=mesh.mesh2d,
)
da_cols = xu.UgridDataArray(
    xr.DataArray(
        data=columns,
        dims=[mesh.mesh2d.face_dimension]
    ),
    grid=mesh.mesh2d,
)

# combine everything into a UgridDataSet
ds = xu.UgridDataset(grids=mesh.mesh2d)
ds["rows"] = da_rows
ds["cols"] = da_cols

# make a fancy plot
f, axs = plt.subplots(ncols=2, figsize=(13, 5))
ds.cols.ugrid.plot(ax=axs[0])
ds.rows.ugrid.plot(ax=axs[1])
axs[0].set_title("column indexes")
axs[1].set_title("row indexes")


### make a dummy ufunc and see if we can retrieve attributes of the row as we loop through

In [ ]:
def print_row(row):
    # we'll simply print the row for now
    print(f"We are printing the row")
    print(row)

